# Notebook for nba players stats dataset

In [1]:
import requests
import pandas as pd
import numpy as np
import string
from bs4 import BeautifulSoup

### Scrape data (1). Method to collect all players dataset

In [2]:
def create_players_df(url):
    response = requests.get(url)
    cols = ['Player_idx', 'Player', 'From', 'To', 'Position', 'Height', 'Weight', 'Birth Date', 'Colleges', 'Link']
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', attrs={'class':'sortable', 'data-cols-to-freeze':"1"}).tbody
    df = pd.DataFrame(columns=cols)
    trs = table.find_all('tr')

    for tr in trs:
        tds = tr.find_all('td')
        row = [td.text.replace('\n', '') for td in tds]
        th = str(tr.find('th', attrs={'class':'left', 'data-stat': 'player'}).a)
        # lenght of  <a href="/players/ - 17
        link = th[17:th.find('>')-1]
        player_id = link[3:-5]
        name = th[th.find('>')+1:-4]
        link = url[:-2]+link
        row.insert(0, player_id)
        row.insert(1, name)
        row.append(link)
        df = df.append(pd.Series(row, index=cols), ignore_index=True)
    return df

### Scrape data (2.1). Method to collect each player career stats. OLD

In [3]:
def create_player_stats_df(player_idx, url, table_id):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table_cols = soup.find('table', attrs={'class':'row_summable', 'id':table_id}).thead
    cols = ['Player_idx']
    trs = table_cols.find_all('tr')
    for tr in trs:
        ths = tr.find_all('th')
        row = [th.text for th in ths]
        cols.extend(row)
        
    table = soup.find('table', attrs={'class':'row_summable', 'id':table_id}).tbody
    df = pd.DataFrame(columns=cols)
    trs = table.find_all('tr')

    for tr in trs:
        tds = tr.find_all('td')
        row = [td.text.replace('\n', '') for td in tds]
        try: #player missed all season due injury
            th = str(tr.find('th', attrs={'class':'left', 'data-stat': 'season'}).a)
            season = th[th.find('>')+1:-4]
            row.insert(0, player_idx)
            row.insert(1, season)
            df = df.append(pd.Series(row, index=cols), ignore_index=True)
        except:
            pass        
    return df.replace(r'^\s*$', np.nan, regex=True)

### Scrape data (2.2). Method to collect each player career stats.

In [4]:
def set_index(df, index):
    df.insert(loc=0, column='Player_idx', value=index)
    df.set_index('Player_idx')
    return df.replace(r'^\s*$', np.nan, regex=True)

In [5]:
def create_player_stats_df(player_idx, url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # all others tables are in comments
    other_tables = soup.find_all(string=lambda text: isinstance(text, Comment))
    
    per_36_min = pd.DataFrame()
    season_totals = pd.DataFrame()
    for each in other_tables:
        if 'id="totals"' in each:
            try:
                season_totals = pd.read_html(each)[0]
            except:
                continue
        elif 'id="per_minute"' in each:
            try:
                per_36_min = pd.read_html(each)[0]
            except:
                continue
    
    per_36_min = set_index(per_36_min, player_idx)
    
    season_totals = set_index(season_totals, player_idx)
    
    per_game_table = soup.find_all('table')[0]
    per_game_data = [[cell.text for cell in row.find_all(['th','td'])]
                        for row in per_game_table.find_all('tr')]
    per_game = pd.DataFrame(per_game_data)
    per_game.columns = per_game.iloc[0,:]
    per_game.drop(index=0,inplace=True)
    per_game = per_game.loc[~per_game['Tm'].str.contains('Did Not Play')]
    per_game = set_index(per_game, player_idx)
    
    return per_game, per_36_min, season_totals

### Scrape data (3). Collect all players into csv

In [6]:
players = pd.DataFrame()
url = 'https://www.basketball-reference.com/players/'

letters = list(string.ascii_lowercase)
for letter in letters:
    players = players.append(create_players_df(url+str(letter)))
    
players.set_index('Player_idx')
players = players.sort_values(by=['Player_idx'])
players.to_csv('all_players.csv',index=False)
players = pd.DataFrame()

### Scrape data (3.1). Collect all players career stats into csv. OLD

In [7]:
players_stats = pd.DataFrame()
df = pd.read_csv('all_players.csv')
players_df_for_stats= df.drop(columns=['Player', 'From', 'To', 'Position', 'Height', 'Weight', 'Birth Date', 'Colleges'])

for i in range(players_df_for_stats.shape[0]):
    players_stats = players_stats.append(create_player_stats_df(players_df_for_stats.iloc[i,0], 
                                                                players_df_for_stats.iloc[i,1], 
                                                                'per_game'))

players_stats.set_index('Player_idx')
players_stats.to_csv('players_stats_per_game.csv',index=False)

players_stats = pd.DataFrame()

### Scrape data (3.2). Collect all players career stats into csvs.

In [8]:
players_df_for_stats = pd.read_csv('all_players.csv')
per_game = pd.DataFrame()
per_36_min = pd.DataFrame() 
season_totals = pd.DataFrame()

for i in range(players_df_for_stats.shape[0]):
    df, df1, df2 = create_player_stats_df(players_df_for_stats.iloc[i,0], players_df_for_stats.iloc[i,-1])
    per_game = per_game.append(df)
    per_36_min = per_36_min.append(df1)
    season_totals = season_totals.append(df2)

per_game = per_game[per_game.Age.notna()]
per_game.to_csv('per_game.csv',index=False)
per_36_min = per_36_min[per_36_min.Age.notna()]
per_36_min.to_csv('per_36_min.csv',index=False)
season_totals = season_totals[season_totals.Age.notna()]
season_totals.to_csv('season_totals.csv',index=False)
lst = [players_df_for_stats, per_game, per_36_min, season_totals]
del lst

### Scrape data (3.3). Collect players born countries except USA.

In [9]:
response = requests.get('https://www.basketball-reference.com/friv/birthplaces.fcgi')
soup = BeautifulSoup(response.content, 'html.parser')
countries = pd.DataFrame()
table = soup.find_all('div')[22] ## table number

countries = pd.DataFrame([cell.text for cell in table.find_all(['p'])])
countries.rename(columns={countries.columns[0]: 'Country'}, inplace=True)
countries['Count'] = countries['Country'].str[:-1].str.split('(').str[1].astype('int')
countries['Country'] = countries['Country'].str.split('(').str[0].str[:-1]

df = pd.read_csv('countries_def.csv')
countries = countries.merge(df, how='inner', left_on=['Country'], right_on = ['Country'])
countries['Country'].replace({'Russian Federation': 'Russia'}, inplace=True)
countries.set_index('Country')
countries.to_csv('countries.csv', index=False)

# Data cleaning

### Data cleaning (1). Remove all seasons before three point era

In [10]:
def remove_old_seasons_with_missing_data(df):
    df = df.loc[df['Lg'] == 'NBA']
    three_point_year = 1979
    df = df.loc[df.loc[ : , 'Season'].str[:4].astype(int) >= three_point_year]
    return df

In [11]:
per_game = pd.read_csv('per_game.csv')
per_36_min = pd.read_csv('per_36_min.csv')
season_totals = pd.read_csv('season_totals.csv')
season_totals = season_totals[season_totals.Age.notna()]
per_36_min = per_36_min[per_36_min.Age.notna()]

per_game = remove_old_seasons_with_missing_data(per_game)
per_36_min = remove_old_seasons_with_missing_data(per_36_min)
season_totals = remove_old_seasons_with_missing_data(season_totals)

### Data cleaning (2). Remove seasons data when player play for more than one team

In [12]:
def remove_dublicated_seasons_and_fill_null_columns(df, size):
    #remove seasons data when player play for more than one team, and change TOT to team1/team2/..teamN
    df['CountTeamsPerSeason'] = df.groupby(['Player_idx','Season'])['Tm'].transform('count')

    grouped_teams = df.groupby(['Player_idx','Season']).apply(lambda x: '/'.join(x.Tm)).to_frame()
    grouped_teams = grouped_teams.replace(to_replace = 'TOT/', value = '', regex = True)
    df = df.merge(grouped_teams, how='inner', left_on=['Player_idx','Season'], right_on = ['Player_idx','Season'])
    df.rename(columns={df.columns[size+1]: "Teams"}, inplace = True)
    df['Tm'] = np.where(df['Tm'] == 'TOT', df['Teams'], df['Tm'])
    df.drop(df[(df['Tm'].str.len() == 3) & (df['CountTeamsPerSeason'] > 1)].index, inplace = True)
    df = df.drop(df.columns[size:], axis=1)

    #fill all null columns
    df.fillna({'GS': 0, 'FG%': 0, '3P%': 0, '2P%': 0, 'eFG%': 0, 'FT%': 0}, inplace=True)
    return df

In [13]:
per_game = remove_dublicated_seasons_and_fill_null_columns(per_game, 31)
per_36_min = remove_dublicated_seasons_and_fill_null_columns(per_36_min, 30)
season_totals = remove_dublicated_seasons_and_fill_null_columns(season_totals, 31)

### Data cleaning (3.1).  Players stats per 36 minutes OLD

In [15]:
df_per_36_min = pd.read_csv('players_stats_per_game_filtered.csv')
df_per_36_min['Rate'] = (df_per_36_min['MP'].astype(float)/36).round(2)

df_per_36_min = df_per_36_min.loc[df_per_36_min['Rate'] > 0]
df_per_36_min = df_per_36_min.drop(['eFG%'], axis=1)
df_per_36_min.iloc[:, 9:30] = df_per_36_min.iloc[:, 9:30].div(df_per_36_min.Rate, axis = 0).round(2)
#df_per_36_min.to_csv('players_stats_per_36_minutes.csv',index=False)
df_per_36_min = pd.DataFrame()

### Data cleaning (3.2). Only players who have played 500 or more matches remain

In [16]:
def final_filter_for_data_analysis(df, size):
    # try filter all players that play 500 and more games per career
    df['TotalGames'] = df.groupby(['Player_idx'])['G'].transform('sum')
    df = df.loc[df['TotalGames'] >= 500]
    df = df.drop(df.columns[size:], axis=1)
    return df

In [17]:
per_game = final_filter_for_data_analysis(per_game, 31)
per_36_min = final_filter_for_data_analysis(per_36_min, 30)
season_totals = final_filter_for_data_analysis(season_totals, 31)

per_game = per_game.sort_values(by=['Player_idx', 'Season'])
per_36_min = per_36_min.sort_values(by=['Player_idx', 'Season'])
season_totals = season_totals.sort_values(by=['Player_idx', 'Season'])

per_game.to_csv('per_game.csv',index=False)
per_36_min.to_csv('per_36_min.csv',index=False)
season_totals.to_csv('season_totals.csv',index=False)